Launch interactive version: 👉👉👉 [![Try ``dyce``](https://jupyterlite.readthedocs.io/en/latest/_static/badge.svg)](https://posita.github.io/dyce-notebooks/lab?path=stack-exchange%2Fironsworn-in-the-dark-200325%2Fironsworn_in_the_dark.ipynb) 👈👈👈 *[[source](https://github.com/posita/dyce-notebooks/tree/main/notebooks/stack-exchange/ironsworn-in-the-dark-200325)]*

## [``dyce``](https://posita.github.io/dyce/) solution to [“Ironsworn meets BitD dice mechanic (with a twist)”](https://rpg.stackexchange.com/a/200357/71245)

Once viewing this notebook in Jupyter Lab, select ``Run All Cells`` from the ``Run`` menu above.

In [1]:
# Install additional requirements if necessary
try:
    import anydyce, ipywidgets
except ImportError:
    requirements = ["anydyce~=0.1.4", "ipywidgets"]
    try:
        import piplite
        await piplite.install(requirements)
    except ImportError:
        import pip
        pip.main(["install"] + requirements)

In [2]:
from dyce import P
from dyce.p import RollT

def ironsworn_bitd(action_roll: RollT, challenge_roll: RollT) -> int:
    assert len(challenge_roll) == 2
    max_action = max(action_roll, default=0)
    basic_successes = sum(1 for c in challenge_roll if c < max_action)
    
    num_max_actions = sum(1 for a in action_roll if a == max_action)
    max_challenge = max(challenge_roll)
    
    if max_action > max_challenge:
        # We count everything but the first max action
        num_offsetting_challenges = 1
    elif max_action < max_challenge:
        # We can't have any match successes in this scenario, so we offset all max
        # actions
        num_offsetting_challenges = num_max_actions
    else:  # max_challenge == max_action
        # We offset the number of max actions by the number of max challenges, but in
        # no case more than the number of max actions
        num_offsetting_challenges = min(
            sum(1 for c in challenge_roll if c == max_action),
            num_max_actions,
        )

    match_successes = num_max_actions - num_offsetting_challenges
    return basic_successes + match_successes

h = P.foreach(ironsworn_bitd, action_roll=4@P(6), challenge_roll=2@P(6))
print(h.format(scaled=True))

avg |    1.66
std |    0.92
var |    0.85
  0 |  10.95% |############
  1 |  29.67% |#################################
  2 |  43.74% |##################################################
  3 |  13.53% |###############
  4 |   1.99% |##
  5 |   0.12% |


In [3]:
from anydyce import BreakoutType, jupyter_visualize

jupyter_visualize(
    [
        (f"{a}d6 action vs 2d6 challenge", P.foreach(ironsworn_bitd, action_roll=a @ P(6), challenge_roll=2 @ P(6)))
        for a in range(1, 6)
    ],
    default_breakout_type=BreakoutType.BURST,
)